In [1]:
#!fsharp
#!value --from-file input.txt --name input

In [1]:
#!fsharp
#!share input --from value
let lines = input.Trim().Split('\n')

In [1]:
#!fsharp
type Operator = Add | Mul with 
  member this.Apply = 
    match this with
    | Add -> (+)
    | Mul -> (*)

type Parenthesis = Left | Right

type Token =
  | Literal of int64
  | Op of Operator
  | Paren of Parenthesis

let (|Int|_|) n = 
  try n |> string |> int64 |> Some with _ -> None

let tokenize = 
  Seq.choose (
    function
    | '*' -> Op Mul |> Some
    | '+' -> Op Add |> Some
    | '(' -> Paren Left |> Some
    | ')' -> Paren Right |> Some
    | Int i -> Literal i |> Some
    | _ -> None
  ) >> Seq.toList

let splitWhile f list =
  match List.tryFindIndex (not << f) list with
  | Some i -> List.take i list, List.skip i list
  | None -> list, []

let evaluate precedence exp = 
  let rec shunt opStack tokens = 
    match (tokens, opStack) with
    | Op o as op :: rest , ops::more -> 
      let (popped, remaining) = ops |> splitWhile (fun c -> precedence c >= precedence o) 
      List.map Op popped @ shunt ((o::remaining)::more) rest
    | Paren Left :: rest, stack -> shunt ([]::stack) rest
    | Paren Right :: rest, ops::stack -> List.map Op ops @ shunt stack rest
    | n :: rest , stack -> n::(shunt stack rest)
    | [], stack -> List.collect (List.map Op) stack
  let rec evalRPN stack tokens =
    match (tokens, stack) with
    | Op o::t, i::j::r -> evalRPN (o.Apply i j::r) t 
    | Literal i :: t, s -> evalRPN (i::s) t
    | [], [i] -> i
    | e -> failwith <| sprintf "%A" e

  exp
  |> tokenize
  |> shunt [[]]
  |> evalRPN []

lines |> Seq.sumBy (evaluate (fun _ -> 0)) |> printf "With no precedence: %i"
lines |> Seq.sumBy (evaluate (function Add -> 1 | Mul -> 0)) |> printf "With + > * precedence: %i"



With no precedence: 

1402255785165

With + > * precedence: 

119224703255966